In [1]:
# подключим необходимые библиотеки
# Единицы измерения
from pint import UnitRegistry
ureg = UnitRegistry()

# Универсальная газовая постоянная
from scipy.constants import R
print(R)
# Единица измерения для универсальной газовой постоянной
R_UOM = ureg.joule / (ureg.mol * ureg.kelvin)
R_UOM

8.314462618


<Unit('joule / kelvin / mole')>





# Газопроницаемость

Оценка газопроницаемости тканей с двусторонним покрыттием (резиновым или ПВХ) для проекта СТО ВГУ ОТУ.

Будет проведена попытка оценки двумя методами:

- первый - на основании установленного требования по ГОСТ 8973 - 0,002 м² / (Па / с)
- второй - на основании установленного требования к уменьшению давления в ВГУ - 10 % за 6 часов

## Исходные данные

### Газопроницаемость реальных материалов

Газопроницаемость ПВХ-ткани в открытых источниках найти не удалось.

Газопроницаемость ПВХ

 Газ | Значение | Единица измерения | Ссылка
:---:|:--------:|:-----------------:|:-------:
Кислород | 0.198 - 0.790 | мл \* м / (м² \* сутки * Па) | [ссылка](https://www.quora.com/How-long-can-a-human-survive-in-an-inflatable-plastic-bubble-with-a-2-meter-diameter)
Кислород | 3100 | см³ \* микрометр / (м² \* сутки * бар) | [ссылка]( http://exclusive.multibriefs.com/content/novel-medical-plastics-packaging-material-and-process-advances/engineering)


Газопроницаемость других материалов (для сравнения)

 Материал | Газ | Значение | Единица измерения | Ссылка | Примечание
:--------:|:---:|:--------:|:-----------------:|:------:|:-----------:
Поливинилиденхлорид | Кислород | 2.6 | см³ \* 0.2 микрометра / (м² \* сутки * атмосферу) | [ссылка](https://www.quora.com/How-long-can-a-human-survive-in-an-inflatable-plastic-bubble-with-a-2-meter-diameter) | Указано, что хороший барьер
Поливинилиденхлорид | Кислород | 0.001 - 0.03 | мл \* м / (м² \* сутки * Па) | [ссылка](https://www.quora.com/How-long-can-a-human-survive-in-an-inflatable-plastic-bubble-with-a-2-meter-diameter)




In [2]:
# Проверим исходные данные, полученные из разных источников
# Сравнить по самим данным достаточно сложно, т.к. единицы измерения очень отличаются
# для перевода воспользуемся математическим пакетом (автоматически переведем единицы измерения) pint
# Единица измерения первой строки таблицы "Газопроницаемость ПВХ"
p_pvc1_UOM = ureg.milliliter * ureg.meter / (ureg.meter * ureg.meter * ureg.day * ureg.pascal)     # UOM - unit of measure
p_pvc1_UOM

<Unit('milliliter / day / meter / pascal')>

In [3]:
# Единица измерения второй строки таблицы "Газопроницаемость ПВХ"
p_pvc2_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter * ureg.micrometer / (ureg.meter * ureg.meter * ureg.day * ureg.bar)
p_pvc2_UOM

<Unit('centimeter ** 3 * micrometer / bar / day / meter ** 2')>

In [4]:
# Перводим значение второй строки в единицы измерения первой строки
permiability_PVC_2 = (3100 * p_pvc2_UOM).to(p_pvc1_UOM)
permiability_PVC_2

3.1000000000000006e-08 <Unit('milliliter / day / meter / pascal')>

In [5]:
permiability_PVC_1 = 0.198 * p_pvc1_UOM
permiability_PVC_1

0.198 <Unit('milliliter / day / meter / pascal')>

In [6]:
permiability_PVC_1 / permiability_PVC_2

6387096.774193548 <Unit('dimensionless')>

In [7]:
# Единица измерения первой строки таблицы "Газопроницаемость других материалов"
p_1_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter * 0.2 * ureg.micrometer / (ureg.meter * ureg.meter * ureg.day * ureg.atmosphere) # UOM - unit of measure
p_1_UOM

0.2 <Unit('centimeter ** 3 * micrometer / day / meter ** 2 / standard_atmosphere')>

In [8]:
# Единица измерения второй строки таблицы "Газопроницаемость других материалов"
p_2_UOM = ureg.milliliter * ureg.meter / (ureg.meter * ureg.meter * ureg.day * ureg.pascal)
p_2_UOM

<Unit('milliliter / day / meter / pascal')>

In [9]:
# Переводим значение первой строки в единицы измерения второй строки
permiability_PVdC_1 = (2.6 * p_1_UOM).to(p_2_UOM)
permiability_PVdC_1

5.1320009869232674e-12 <Unit('milliliter / day / meter / pascal')>

In [10]:
# Значение второй строки
permiability_PVdC_2 = 0.001 * p_2_UOM
permiability_PVdC_2

0.001 <Unit('milliliter / day / meter / pascal')>

In [11]:
# Сравним значение первой строки со значением второй строки в одинаковых единицах измерения
permiability_PVdC_2 / permiability_PVdC_1

194855769.23076922 <Unit('dimensionless')>

In [12]:
f'Первое значение {permiability_PVdC_1.magnitude:.2e} в {(permiability_PVdC_2 / permiability_PVdC_1).magnitude:.2e} раз меньше второго значения {permiability_PVdC_2.magnitude}'

'Первое значение 5.13e-12 в 1.95e+08 раз меньше второго значения 0.001'

**Вывод**

Ориентироваться на показания, полученные из открытых источников нельзя:

- не известно по какой методике проведен контроль;
- не известна достоверность результатов (данные отличаются в несколько раз).

## Метод 1

В первом методе выведем значения газопроницаемости по двум оставшимся ГОСТам (ГОСТ 23553 и ГОСТ 53656.2) по значению, которое установлено сейчас для первого ГОСТа (ГОСТ 8973)

Значение воздухопроницаемости по ГОСТ 8973 - 0,002 м² / (Па / с)

### ГОСТ 8973

In [13]:
gost8973_UOM = ureg.meter * ureg.meter / (ureg.pascal * ureg.second)
gost8973_permiability = 0.02 * gost8973_UOM

### ГОСТ 23553

### ГОСТ Р 53656.2

In [14]:
# Основные единицы измерения по ГОСТ 53656 для
# Р - газопроницаемость (скорость проникновения газа на обратную единицу толщины - чем толще, тем ниже газопроницаемость)
# GTR - скорость проникновения газа (отличается тем, что не зависит от толщины)
gost53656_P_UOM = ureg.mol * ureg.meter / (ureg.meter * ureg.meter * ureg.second * ureg.pascal)
gost53656_GTR_UOM = ureg.mol / (ureg.meter * ureg.meter * ureg.second * ureg.pascal)

gost53656_P_alt_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter * ureg.millimeter / (ureg.meter * ureg.meter * ureg.day * ureg.atmosphere)
gost53656_GTR_alt_UOM = ureg.centimeter * ureg.centimeter * ureg.centimeter / (ureg.meter * ureg.meter * ureg.day * ureg.atmosphere)

In [15]:
# Перевод значения по ГОСТ 8973 в значения по ГОСТ 53656 (альтернативные единицы измерения, приведенные в п. А.6.2)
# TODO Подумать, надо ли разделять только на кислород или считать как для всего воздуха
gost53656_permiability_alt = (gost8973_permiability).to(gost53656_P_alt_UOM)
gost53656_permiability_alt

1.7508959999999997e+17 <Unit('centimeter ** 3 * millimeter / day / meter ** 2 / standard_atmosphere')>

In [17]:
# Переведем в основные единицы измерения по ГОСТ 53656
# Чтобы перевести необходимо привести к нормальным условиям
gost53656_permiability = gost53656_permiability_alt * 10**5 * ureg.pascal / (R * R_UOM * 293 * ureg.kelvin)
gost53656_permiability.to(gost53656_P_UOM)

0.8209717067914181 <Unit('mole / meter / pascal / second')>

## Метод 2

На основании установленного требования к уменьшению давления в ВГУ - 10 % за 6 часов

### ГОСТ 23553

### ГОСТ 53656.2

## Итоги

In [19]:
import pandas as pd

In [27]:
dates = pd.DataFrame(data={'1' : ['23'], '2' : ['34'], '3' : ['34']})
dates

,1,2,3
0,23,34,34
